In [45]:
import os

RESULTS_DIR = "../results"
METRICS_DIR = os.path.join(RESULTS_DIR, "metrics")
VIS_DIR = os.path.join(RESULTS_DIR, "visualization")

os.makedirs(METRICS_DIR, exist_ok=True)
os.makedirs(VIS_DIR, exist_ok=True)

print("Folders ready:", METRICS_DIR, VIS_DIR)


Folders ready: ../results/metrics ../results/visualization


In [46]:
from tensorflow.keras.models import load_model

baseline_model = load_model("models/baseline.keras")
resnet_model   = load_model("models/resnet50.keras")
eff_model      = load_model("models/effnetB0.keras")


import pickle

with open("models/history_baseline.pkl", "rb") as f:
    history_baseline = pickle.load(f)

with open("models/history_resnet.pkl", "rb") as f:
    history_res_ft = pickle.load(f)

with open("models/history_effnet.pkl", "rb") as f:
    history_eff_ft = pickle.load(f)


In [47]:
root_path = "../data/processed"

train_dir = os.path.join(root_path, "train" \
"")
val_dir   = os.path.join(root_path, "val")
test_dir  = os.path.join(root_path, "test")

img_size = (224, 224)
batch_size = 32
seed = 42

train_ds = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    image_size=img_size,
    batch_size=batch_size,
    label_mode="categorical",
    shuffle=True,
    seed=seed,
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    val_dir,
    image_size=img_size,
    batch_size=batch_size,
    label_mode="categorical",
    shuffle=False,
)

test_ds = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    image_size=img_size,
    batch_size=batch_size,
    label_mode="categorical",
    shuffle=False,
)

class_names = train_ds.class_names
num_classes = len(class_names)
print("Classes:", class_names)

AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(AUTOTUNE)
val_ds   = val_ds.cache().prefetch(AUTOTUNE)
test_ds  = test_ds.cache().prefetch(AUTOTUNE)


Found 4426 files belonging to 5 classes.
Found 948 files belonging to 5 classes.
Found 949 files belonging to 5 classes.
Classes: ['F0', 'F1', 'F2', 'F3', 'F4']


In [48]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

from sklearn.metrics import (
    confusion_matrix,
    classification_report,
    roc_curve,
    auc
)


In [49]:
# Training curves
def plot_training_curves(history, model_name):
    plt.figure(figsize=(12,5))

    # Accuracy
    plt.subplot(1,2,1)
    plt.plot(history["accuracy"])
    plt.plot(history["val_accuracy"])
    plt.title(f"{model_name} Accuracy")
    plt.legend(["Train", "Val"])

    # Loss
    plt.subplot(1,2,2)
    plt.plot(history["loss"])
    plt.plot(history["val_loss"])
    plt.title(f"{model_name} Loss")
    plt.legend(["Train", "Val"])

    out_path = os.path.join(VIS_DIR, f"{model_name}_training_curves.png")
    plt.savefig(out_path, dpi=150)
    plt.close()
    print("Saved:", out_path)


In [50]:
# Confusion matrix + predictions
def evaluate_model(model, dataset, class_names, model_name, split_name="test"):
    y_true = []
    y_pred = []

    for x, y in dataset:
        preds = model.predict(x)
        y_pred.extend(np.argmax(preds, axis=1))
        y_true.extend(np.argmax(y.numpy(), axis=1))

    cm = confusion_matrix(y_true, y_pred)

    #  Plot CM 
    plt.figure(figsize=(7,6))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
                xticklabels=class_names, yticklabels=class_names)
    plt.title(f"{model_name} Confusion Matrix ({split_name})")
    
    out_path = os.path.join(VIS_DIR, f"{model_name}_{split_name}_cm.png")
    plt.savefig(out_path, dpi=150)
    plt.close()
    print("Saved:", out_path)

    return np.array(y_true), np.array(y_pred)


In [51]:
# Classification report
def save_classification_report(y_true, y_pred, class_names, model_name, split_name="test"):
    report = classification_report(
        y_true, y_pred,
        target_names=class_names,
        output_dict=True
    )
    df = pd.DataFrame(report).T

    out_path = os.path.join(METRICS_DIR, f"{model_name}_{split_name}_classification_report.csv")
    df.to_csv(out_path)

    print("Saved:", out_path)
    return df


In [52]:
# ROC curve function
def plot_roc_curves(model, dataset, class_names, model_name):

    all_preds, all_labels = [], []

    for x, y in dataset:
        all_preds.append(model.predict(x))
        all_labels.append(y.numpy())

    preds = np.concatenate(all_preds)
    labels = np.concatenate(all_labels)  # one-hot
    n_classes = labels.shape[1]

    plt.figure(figsize=(8,7))

    for i in range(n_classes):
        fpr, tpr, _ = roc_curve(labels[:, i], preds[:, i])
        roc_auc = auc(fpr, tpr)
        plt.plot(fpr, tpr, label=f"{class_names[i]} (AUC={roc_auc:.2f})")

    plt.plot([0,1], [0,1], 'k--')
    plt.title(f"{model_name} ROC Curves")
    plt.xlabel("FPR")
    plt.ylabel("TPR")
    plt.legend()

    out_path = os.path.join(VIS_DIR, f"{model_name}_roc.png")
    plt.savefig(out_path, dpi=150)
    plt.close()
    print("Saved:", out_path)


In [53]:
# GRAD-CAM generation
def generate_gradcam(model, img_array, layer_name):
    grad_model = tf.keras.models.Model(
        [model.inputs],
        [model.get_layer(layer_name).output, model.output]
    )

    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        pred_class = tf.argmax(predictions[0])
        loss = predictions[:, pred_class]

    grads = tape.gradient(loss, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0,1,2))
    conv_outputs = conv_outputs[0]

    heatmap = tf.reduce_sum([
        conv_outputs[:, :, i] * pooled_grads[i]
        for i in range(pooled_grads.shape[-1])
    ], axis=0)

    heatmap = np.maximum(heatmap, 0)
    heatmap /= (np.max(heatmap) + 1e-8)
    return heatmap


In [54]:
# Grad-CAM overlay
def save_gradcam_overlay(img, heatmap, model_name, sample_id):
    plt.figure(figsize=(5,5))
    plt.imshow(img)
    plt.imshow(heatmap, cmap="jet", alpha=0.45)
    plt.axis("off")

    out_path = os.path.join(VIS_DIR, f"{model_name}_gradcam_{sample_id}.png")
    plt.savefig(out_path, dpi=150)
    plt.close()

    print("Saved:", out_path)


In [55]:
# Running for all models
# Example variables
# baseline_model, resnet_model, effnet_model
# train_ds, val_ds, test_ds
# class_names

models = {
    "baseline": baseline_model,
    "resnet50": resnet_model,
    "efficientnetB0": eff_model
}

histories = {
    "baseline": history_baseline,
    "resnet50": history_res_ft,
    "efficientnetB0": history_eff_ft
}

for name, model in models.items():

    print(name.upper())

    # 1. Training curves
    plot_training_curves(histories[name], name)

    # 2. Confusion matrix + predictions
    y_true, y_pred = evaluate_model(model, test_ds, class_names, name)

    # 3. Classification report
    save_classification_report(y_true, y_pred, class_names, name)

    # 4. ROC curves
    plot_roc_curves(model, test_ds, class_names, name)


BASELINE
Saved: ../results/visualization/baseline_training_curves.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━

2025-11-24 23:50:51.697549: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
/opt/anaconda3/envs/CB/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/opt/anaconda3/envs/CB/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/opt/anaconda3/envs/CB/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no 

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━

2025-11-24 23:50:53.756146: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 253ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 230ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 229ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 230ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 229ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 227ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 231ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 228ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 241ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 238ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 257ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 239ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 228ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 227ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 278ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 244ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 230ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 231ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 234ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 228ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 231ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 228ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 229ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 228ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 

...

2025-11-24 23:51:13.446162: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 321ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 179ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 166ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 165ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 167ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 160ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 158ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 159ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 164ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 160ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 158ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 158ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 158ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 160ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 162ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 164ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 158ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 159ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 156ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 162ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 155ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 157ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 156ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 159ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 

2025-11-24 23:51:28.728366: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 332ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 220ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 169ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 175ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 160ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 161ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 171ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 167ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 168ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 158ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 159ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 166ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 160ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 163ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 159ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 158ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 159ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 158ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 159ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 158ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 160ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 158ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 166ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 170ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 164ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 

2025-11-24 23:51:34.424074: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [56]:
import random

for name, model in models.items():
    print("\nGenerating Grad-CAM for:", name)

    # pick 5 random samples
    i = 0
    for img, lbl in test_ds.unbatch().take(50):
        if i == 5:
            break

        img_array = tf.expand_dims(img, 0)
        heatmap = generate_gradcam(model, img_array, layer_name=model.layers[-5].name)

        save_gradcam_overlay(img.numpy().astype("uint8"), heatmap, name, i)
        i += 1



Generating Grad-CAM for: baseline


/opt/anaconda3/envs/CB/lib/python3.12/site-packages/keras/src/models/functional.py:241: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: [['input_layer_2']]
Received: inputs=['Tensor(shape=(1, 224, 224, 3))']
  warnings.warn(msg)


Saved: ../results/visualization/baseline_gradcam_0.png
Saved: ../results/visualization/baseline_gradcam_1.png
Saved: ../results/visualization/baseline_gradcam_2.png
Saved: ../results/visualization/baseline_gradcam_3.png
Saved: ../results/visualization/baseline_gradcam_4.png

Generating Grad-CAM for: resnet50


/opt/anaconda3/envs/CB/lib/python3.12/site-packages/keras/src/models/functional.py:241: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: [['input_layer_4']]
Received: inputs=['Tensor(shape=(1, 224, 224, 3))']
  warnings.warn(msg)
2025-11-24 23:51:36.060856: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: INVALID_ARGUMENT: Invalid reduction dimension (2 for input with 2 dimension(s)


InvalidArgumentError: {{function_node __wrapped__Mean_device_/job:localhost/replica:0/task:0/device:GPU:0}} Invalid reduction dimension (2 for input with 2 dimension(s) [Op:Mean]

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os

def find_last_conv_layer(model):
    """Find the last Conv2D layer in a Keras model, skipping preprocessing/augmentation layers."""
    for layer in reversed(model.layers):
        if isinstance(layer, tf.keras.layers.Conv2D):
            return layer
        # For nested models or Sequential blocks
        if hasattr(layer, 'layers'):
            conv = find_last_conv_layer(layer)
            if conv is not None:
                return conv
    raise ValueError(f"No Conv2D layers found in model {model.name}")

def generate_gradcam(model, img_array, layer_name):
    grad_model = tf.keras.models.Model(
        [model.inputs],
        [model.get_layer(layer_name).output, model.output]
    )
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        pred_class = tf.argmax(predictions[0])
        loss = predictions[:, pred_class]

    grads = tape.gradient(loss, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0,1,2))
    conv_outputs = conv_outputs[0]

    heatmap = tf.reduce_sum(conv_outputs * pooled_grads, axis=-1)
    heatmap = np.maximum(heatmap, 0)
    heatmap /= (np.max(heatmap) + 1e-8)
    return heatmap

def save_gradcam_overlay(img, heatmap, model_name, sample_id, save_dir="gradcam_outputs"):
    os.makedirs(save_dir, exist_ok=True)
    plt.figure(figsize=(5,5))
    plt.imshow(img, cmap="gray")
    plt.imshow(heatmap, cmap="jet", alpha=0.45)
    plt.axis("off")
    out_path = os.path.join(save_dir, f"{model_name}_gradcam_{sample_id}.png")
    plt.savefig(out_path, dpi=150)
    plt.close()
    print("Saved:", out_path)

# Generate Grad-CAMs for all models
for name, model in models.items():
    print("\nGenerating Grad-CAM for:", name)
    last_conv_layer = find_last_conv_layer(model)
    print(f"Using last conv layer: {last_conv_layer.name}")

    i = 0
    for img, lbl in test_ds.unbatch().take(50):
        if i == 5:
            break
        img_array = tf.expand_dims(img, 0)
        heatmap = generate_gradcam(model, img_array, layer_name=last_conv_layer.name)
        save_gradcam_overlay(img.numpy().astype("uint8"), heatmap, name, i)
        i += 1



Generating Grad-CAM for: baseline
Using last conv layer: conv2d_5
Saved: gradcam_outputs/baseline_gradcam_0.png
Saved: gradcam_outputs/baseline_gradcam_1.png
Saved: gradcam_outputs/baseline_gradcam_2.png
Saved: gradcam_outputs/baseline_gradcam_3.png
Saved: gradcam_outputs/baseline_gradcam_4.png

Generating Grad-CAM for: resnet50
Using last conv layer: conv5_block3_3_conv


ValueError: No such layer: conv5_block3_3_conv. Existing layers are: ['input_layer_4', 'data_augmentation', 'resnet50', 'global_average_pooling2d_2', 'dropout_8', 'dense_4', 'dropout_9', 'dense_5'].